# Import modules

In [1]:
import torch
print("Torch version:",torch.__version__)
print("Is CUDA enabled?",torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.randn(1).cuda())

import importlib
import subprocess

try:
    importlib.import_module('pygrpm')
except ImportError:
    subprocess.check_call(["pip", "install", "git+https://github.com/johndef64/GRPM_system.git"])
    
from pygrpm import *

# GET Datasets

In [3]:
### GET Datasets ###
if not os.path.exists('grpm_dataset/grpm_dataset.parquet'):
    get_and_extract('grpm_dataset', record_id='14052302')
    get_and_extract('nutrigenetic_dataset', record_id='14052302')

# LOAD Datasets

In [2]:
### LOAD Datasets ###
pcg_grpm, rna_grpm, pseudo_grpm = grpm_importer()
grpm_nutrigen, grpm_nutrigen_int, grpm_nutrigen_int_gwas = nutrig_importer()

display(grpm_nutrigen_int)

Importing time:  0:00:07.151996
pcg: 776.19 MB
rna: 58.18 MB
pseudo: 1.93 MB
nutrigen dataset: 87.13 MB
nutrigen dataset filtered: 53.62 MB
nutrigen gwas dataset: 20.56 MB


,gene,rsid,pmid,mesh,topic,interest_index
0,FTO,rs9972653,32393786,Body Mass Index,General Nutrition,1.00000
1,FTO,rs9972653,32393786,"Diabetes Mellitus, Type 2",General Nutrition,1.00000
2,FTO,rs9972653,32393786,Diet,General Nutrition,1.00000
3,FTO,rs9972653,33128006,Body Mass Index,General Nutrition,1.00000
4,FTO,rs9972653,33128006,Cardiovascular Diseases,General Nutrition,1.00000
...,...,...,...,...,...,...
1171244,FADS1,rs174545,23221573,Lipid Metabolism,Xenobiotics Metabolism,0.01255
1171245,FADS1,rs174545,30120404,Liver,Xenobiotics Metabolism,0.01255
1171246,FADS1,rs174545,30120404,Non-alcoholic Fatty Liver Disease,Xenobiotics Metabolism,0.01255
1171247,FADS1,rs174544,20565855,Lipid Metabolism,Xenobiotics Metabolism,0.01255


# SHOW Stats

In [ ]:
%%time
pcg_grpm_stats = get_stats(pcg_grpm, group_by = 'gene')
display(pcg_grpm_stats)

In [ ]:
%%time
grpm_nutrigen_stats = get_stats(grpm_nutrigen, group_by = 'gene')
display(grpm_nutrigen_stats)

# QUERY GRPM Dataset 

## MeSH Query Example

In [3]:
# LOAD MeSH

grpm_mesh= import_grpm_mesh()
grpm_mesh.head()

GRPM MeSH count: 21705


,Preferred Label,Class ID,Synonyms,Definitions
0,Electronic Health Records,http://purl.bioontology.org/ontology/MESH/D057286,"Electronic Medical Record|Medical Record, Elec...",Media that facilitate transportability of pert...
1,Consent Forms,http://purl.bioontology.org/ontology/MESH/D032962,Informed Consent Documents|Informed Consent Fo...,Documents describing a medical treatment or re...
2,Genealogy and Heraldry,http://purl.bioontology.org/ontology/MESH/D005789,Geneology and Heraldry|Heraldry and Genealogy|...,"Descent of a person, family, or group from an ..."
3,Publications,http://purl.bioontology.org/ontology/MESH/D011642,Publication,Copies of a work or document distributed to th...
4,Pharmaceutical Services,http://purl.bioontology.org/ontology/MESH/D010593,"Pharmaceutical Service|Services, Pharmaceutic|...",Total pharmaceutical services provided by qual...


In [5]:
# Random Query Example
mesh_query =  grpm_mesh['Preferred Label'].drop_duplicates().sample(10).to_list()

# Filter and get unique results
result = query_dataset(pcg_grpm, mesh_query, 'mesh')
display(result)

,gene,type,rsid,pmid,mesh,qualifier,major
178386,KRT23,PCG,rs140407470,21874024,CD28 Antigens,physiology,False
222732,MMEL1,PCG,rs3890745,19898481,CD28 Antigens,genetics,True
361776,HLA-DRA,PCG,rs3135392,33058932,Cephalosporins,adverse effects,True
400814,TNF,PCG,rs1800629,25510954,CD28 Antigens,analysis,False
589282,PTPRC,PCG,rs10919563,19898481,CD28 Antigens,genetics,True
...,...,...,...,...,...,...,...
16415706,TNPO3,PCG,rs10488631,31030958,CD28 Antigens,antagonists & inhibitors,False
16416142,TNPO3,PCG,rs10488631,27092776,CD28 Antigens,genetics,True
16416202,TNPO3,PCG,rs10488631,34017081,CD28 Antigens,genetics,True
16488218,ALOX15,PCG,rs34210653,31301373,Turbinates,metabolism,True


## Build MeSH Query 
[CUDA recommended] - In Colab:: load Runtime with GPU 

In [5]:
# LOAD Language Model
MODEL = 'dmis-lab/biobert-v1.1'
model = load_language_model(MODEL)
file_path = 'ref-mesh/GrpmMeshSynEmbeddings_biobert-v1.1.pkl'

# Get MeSH embeddings
test_cuda()
grpm_mesh_embeddings = get_mesh_embeddings(grpm_mesh, model, file_path)

grpm_meshes = grpm_mesh_embeddings['meshes']
mesh_embeddings = grpm_mesh_embeddings['embeddings']

No sentence-transformers model found with name dmis-lab/biobert-v1.1. Creating a new one with mean pooling.


Torch version: 2.6.0+cu118
Is CUDA enabled? True
Importing pretrained embeddings...
Done


In [14]:
# User defined Topic Terms

user_query = "diet ketogenic, diet reducing, diet sodium-restricted, diet, dietary, dietetics, dyslipidemias, eating disorders, feeding and eating disorde, food hypersensitivity, foodborne diseases, gastrointestinal diseases, hypercholesterolemia, hyperglycemia, hyperlipidemias, hyperphagia, hypoglycemia, hypophagia, insulin resistance"  # comma separated list

topic_terms_list = user_query.split(',')
topic_terms = pd.Series(topic_terms_list)

# mesh_query = get_mesh_query(user_query, grpm_meshes, model, mesh_embeddings=mesh_embeddings, threshold=0.90)

# Extract MeSH Query
tab = create_corr_table(topic_terms, grpm_meshes, model, mesh_embeddings_)

threshold = 0.84 # set similarity threshold
mesh_query = tab[tab.similarity >= threshold].list2.to_list()
print('\n\nMeSH Query:', mesh_query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 19/19 [00:03<00:00,  5.59it/s]



MeSH Query: ['Diet, Carbohydrate-Restricted', 'Hyperlipidemias', 'Feeding and Eating Disorders of Childhood', 'Egg Hypersensitivity', 'Endoscopy, Gastrointestinal', 'Hypercholesterolemia', 'Hypertriglyceridemia', 'Hyperlipidemias', 'Hypoglycemic Agents']


MeSH Query: ['Diet', 'Diet', 'Dyslipidemias', 'Hypersensitivity', 'Gastrointestinal Diseases', 'Hypercholesterolemia', 'Hyperglycemia', 'Hyperlipidemias', 'Hyperphagia', 'Hypoglycemia']

## Execute MeSH Query

In [ ]:
# Filter and get unique results
result = query_dataset(pcg_grpm, mesh_query, 'mesh')
display(result)

# QUERY Nutrigenetic Dataset

In [6]:
get_stats(grpm_nutrigen_int, 'topic')

Computing Stats...
runtime:  0:00:01.133999


,gene,rsid,pmid,mesh
,unique,unique,unique,unique
topic,,,,
Cardiovascular Health and Lipid Metabolism,975,41931,66113,521
Diabetes Mellitus Type II and Metabolic Syndrome,603,22270,36198,297
Diet-induced Oxidative Stress,75,2559,10058,60
Eating Behavior and Taste Sensation,211,4252,7241,256
Food Allergies,451,6289,7762,64
Food Intolerances,392,5008,4726,125
General Nutrition,686,26456,44859,397
"Obesity, Weight Control and Compulsive Eating",317,10842,22123,230


## 1. Query by Nutritional Topic 

In [17]:
# Select Topic
topic = "Vitamin and Micronutrients Metabolism and Deficiency-Related Diseases"

# Filter and get unique results
topic_data = query_dataset(grpm_nutrigen_int, [topic], 'topic')
print(f'Displaying "{topic}" topic')
display(topic_data)

Displaying "Vitamin and Micronutrients Metabolism and Deficiency-Related Diseases" topic


,gene,rsid,pmid,mesh,topic,interest_index
967603,VDR,rs987849,34578986,Nutritional Status,Vitamin and Micronutrients Metabolism and Defi...,1.00000
967604,VDR,rs987849,34578986,"Receptors, Calcitriol",Vitamin and Micronutrients Metabolism and Defi...,1.00000
967605,VDR,rs987849,34578986,Vitamin D,Vitamin and Micronutrients Metabolism and Defi...,1.00000
967606,VDR,rs987849,34578986,Vitamin D Deficiency,Vitamin and Micronutrients Metabolism and Defi...,1.00000
967607,VDR,rs987849,34578986,Vitamin D-Binding Protein,Vitamin and Micronutrients Metabolism and Defi...,1.00000
...,...,...,...,...,...,...
1002198,ITPR1,rs121912425,21555639,"Inositol 1,4,5-Trisphosphate Receptors",Vitamin and Micronutrients Metabolism and Defi...,0.01255
1002199,ITPR1,rs121912425,22986007,"Inositol 1,4,5-Trisphosphate Receptors",Vitamin and Micronutrients Metabolism and Defi...,0.01255
1002200,ITPR1,rs121912425,18579805,"Inositol 1,4,5-Trisphosphate Receptors",Vitamin and Micronutrients Metabolism and Defi...,0.01255
1002201,ITPR1,rs121912425,27108798,"Inositol 1,4,5-Trisphosphate Receptors",Vitamin and Micronutrients Metabolism and Defi...,0.01255


In [ ]:
# Get Topic Data Stats
stats = get_stats(topic_data, "gene", gi_sort=True)
stats

### Select Topic on Nutrigenetic-GWAS dataset

In [36]:
# Select Topic on Nutrigenetic-GWAS dataset
topic = "Vitamin and Micronutrients Metabolism and Deficiency-Related Diseases"

topic_data_gwas = query_dataset(grpm_nutrigen_int_gwas, [topic], 'GRPM_TOPIC')
display(topic_data_gwas.head())
# Get Topic Data Stats
stats = get_stats(topic_data_gwas, group_by = "GRPM_GENE")
stats

,GRPM_GENE,GRPM_RSID,GRPM_PMID,GRPM_MESH,GRPM_TOPIC,GRPM_GI,SEMANTIC_SIMILARITY,GWAS_DISEASE/TRAIT,GWAS_MAPPED_TRAIT,GWAS_MAPPED_GENE,GWAS_CONTEXT,GWAS_STRONGEST_SNP-RISK_ALLELE,GWAS_OR-BETA,GWAS_STUDY,GWAS_STUDY_ID
164708,MTHFR,rs1801133,26025547,Vitamin B 12,Vitamin and Micronutrients Metabolism and Defi...,0.64677,0.952817,Vitamin B9 levels,folic acid measurement,MTHFR,missense_variant,rs1801133-A,0.12,The STROMICS genome study: deep whole-genome s...,GCST90319528
164709,MTHFR,rs1801133,33290257,Vitamin B 12,Vitamin and Micronutrients Metabolism and Defi...,0.64677,0.952817,Vitamin B9 levels,folic acid measurement,MTHFR,missense_variant,rs1801133-A,0.12,The STROMICS genome study: deep whole-genome s...,GCST90319528
164710,MTHFR,rs1801133,33799553,Vitamin B 12,Vitamin and Micronutrients Metabolism and Defi...,0.64677,0.952817,Vitamin B9 levels,folic acid measurement,MTHFR,missense_variant,rs1801133-A,0.12,The STROMICS genome study: deep whole-genome s...,GCST90319528
164711,MTHFR,rs1801133,33347560,Vitamin B 12,Vitamin and Micronutrients Metabolism and Defi...,0.64677,0.952817,Vitamin B9 levels,folic acid measurement,MTHFR,missense_variant,rs1801133-A,0.12,The STROMICS genome study: deep whole-genome s...,GCST90319528
164712,MTHFR,rs1801133,17277043,Vitamin B 12,Vitamin and Micronutrients Metabolism and Defi...,0.64677,0.952817,Vitamin B9 levels,folic acid measurement,MTHFR,missense_variant,rs1801133-A,0.12,The STROMICS genome study: deep whole-genome s...,GCST90319528


Computing Stats...
runtime:  0:00:00.161999


,GRPM_RSID,GRPM_PMID,GRPM_MESH,GRPM_TOPIC,GWAS_DISEASE/TRAIT,GWAS_MAPPED_TRAIT,GWAS_MAPPED_GENE,GWAS_CONTEXT,GWAS_STRONGEST_SNP-RISK_ALLELE,GWAS_STUDY,GWAS_STUDY_ID
,unique,unique,unique,unique,unique,unique,unique,unique,unique,unique,unique
GRPM_GENE,,,,,,,,,,,
AMDHD1,1,21,2,1,2,2,1,1,1,1,2
BTD,2,25,1,1,1,1,1,1,2,1,1
CUBN,1,8,1,1,1,1,1,1,1,1,1
CYP24A1,2,24,1,1,1,1,1,2,2,1,1
CYP2R1,1,5,2,1,2,2,1,1,1,2,2
CYP4F2,1,10,1,1,1,1,1,1,1,1,1
GC,7,345,3,1,6,4,1,2,9,8,10
GSTP1,1,484,1,1,1,1,1,1,1,1,1


## 2. Advanced query (use case)

Exploring the genetic determinants of nutritional status involves understanding how genetic variations influence the intake and utilization of micronutrients, impacting nutrient transport, metabolism, and cellular uptake.

"""
Micronutrients such as trace elements and vitamins are important as enzyme cofactors in the metabolism of all cells in the body and therefore key to determining nutritional status. 
"""

Build a composite query:
- Nutritional Status, Mechanisms of Micronutrient Metabolism, and Micronutrient Measurement

In [11]:
from pygrpm import *

# Download and import MeSH Embeddings
grpm_mesh_embeddings = import_mesh_embeddings()

grpm_meshes = grpm_mesh_embeddings['meshes']
mesh_embeddings = grpm_mesh_embeddings['embeddings']

# Define queries using natural language
QUERIES =[
     # 1. **Nutritional Status**:
     ["Measurement of nutritional status", 0.85],
     ["Assess essential micronutrients", 0.84],
     ["Focus on vitamins like vitamin A, D, and B-vitamins.", 0.84],
     ["trace minerals such as iron, zinc, and iodine.", 0.84],

    # 2"**Mechanisms of Micronutrient Metabolism**:
     ["cellular processes for micronutrient absorption.", 0.87],
     ["transport and transformation of nutrients.", 0.84],
     ["nutrients storage mechanisms.", 0.84],
     ["cofactors in nutrient utilization.", 0.84],
     ["homeostasis of nutrients.", 0.84],
]


mesh_query = []
for i in range(len(QUERIES)):
    query =  QUERIES[i]
    print("\n",query)
    meshes = get_mesh_query(query[0], grpm_meshes, model, mesh_embeddings=mesh_embeddings, threshold=query[1])
    mesh_query.extend(meshes)

mesh_query = list(set(mesh_query))

Done

 ['Measurement of nutritional status', 0.85]
Related MeSH: ['Nutritional Support', 'Malnutrition', 'Nutritional Status', 'Lung Volume Measurements', 'Speech Production Measurement', 'Eye Movement Measurements', 'Nutritional Requirements', 'Educational Measurement']

 ['Assess essential micronutrients', 0.84]
Related MeSH: ['Dietary Proteins', 'Micronutrients', 'Malnutrition', 'Vitamin A Deficiency', 'Nutrition Disorders', 'Nutritional Status', 'Nutrients', 'Sports Nutritional Sciences', 'Nutritional Requirements']

 ['Focus on vitamins like vitamin A, D, and B-vitamins.', 0.84]
Related MeSH: ['Vitamin E', 'Cholagogues and Choleretics', 'Anti-Infective Agents, Urinary', 'Anti-Infective Agents, Local', 'Fatty Acids, Omega-3', 'Vitamin B 12', 'Vitamin B Complex', 'Vitamin A', 'Vitamin D', 'Diuretics, Potassium Sparing', 'Vaccines, Virus-Like Particle', 'Amino Acids, Essential', 'Vitamin K 3', 'Vitamin K 2', 'Vitamin B 6', 'Contraceptives, Oral, Hormonal', 'Receptors, OSM-LIF', 'Sali

Get Genes and Variants possibly related to the Query

In [20]:
# Filter and get unique results
query_result = query_dataset(grpm_nutrigen_int, mesh_query, 'mesh')

get_stats(query_result, "gene", gi_sort=True)

Computing Stats...
runtime:  0:00:02.702022


,rsid,pmid,mesh,topic
,unique,unique,unique,unique
gene,,,,
FTO,28,95,16,6
MTHFR,56,614,25,9
VDR,152,376,9,7
TCF7L2,9,56,12,6
HFE,11,100,11,7
...,...,...,...,...
UGT1A3,1,1,1,1
MPO,1,1,1,1


In [26]:
## Select specific genes

# Gene Query 
my_genes = "VDR, PNPLA3, PNPLA3"

# Filter and get unique results
gene_panel = query_dataset(query_result, my_genes.split(','), 'gene')
display(gene_panel)

,gene,rsid,pmid,mesh,topic,interest_index
12321,VDR,rs987849,34578986,Nutritional Status,General Nutrition,0.78441
12323,VDR,rs987849,34578986,Vitamin D,General Nutrition,0.78441
12324,VDR,rs987849,34578986,Vitamin D Deficiency,General Nutrition,0.78441
12327,VDR,rs987849,22046258,Vitamin D,General Nutrition,0.78441
12332,VDR,rs987849,20086113,Vitamin D,General Nutrition,0.78441
...,...,...,...,...,...,...
1043634,VDR,rs10875694,34578986,Nutritional Status,Food Intolerances,0.02603
1043635,VDR,rs10875693,34578986,Nutritional Status,Food Intolerances,0.02603
1043636,VDR,rs10783219,34578986,Nutritional Status,Food Intolerances,0.02603
1043637,VDR,rs10783218,34578986,Nutritional Status,Food Intolerances,0.02603
